In [69]:
import pandas as pd

In [70]:
df=pd.read_csv("/Users/avantikagupta/spotify recommender system /spotify_millsongdata.csv")


In [71]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [72]:
df.head(5)

,artist,song,text
0,Neil Sedaka,I Am A Song,I AM A SONG \r\n \r\nI'm a song and I've wai...
1,Used,Slit Your Own Throat,I gotta take this moment \r\nJust to push you...
2,Hooverphonic,Someone,And through all Alex's wondrous \r\nChildhood...
3,Phil Collins,There's A Place For Us,"'There's a place for us, somewhere a time and ..."
4,Conway Twitty,Call It What You Want To,You tell me that it's over \r\nI'm just someo...


In [73]:
df.tail(5)

,artist,song,text
4995,W.A.S.P.,Running Wild In The Streets,"I'm looking to beat hell, looks like I've got ..."
4996,Queen Latifah,Don't Cry Baby,"Don't cry, don't cry, baby \r\nDon't cry, bab..."
4997,Linkin Park,Not Alone,"I break down, fear is sinking in \r\nThe cold..."
4998,Tori Amos,Famous Blue Raincoat,"It's four in the morning, the end of December ..."
4999,Oasis,Ain't Got Nothin',"They got nothing on me anymore, \r\nThey got ..."


In [74]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [75]:
df['text'][0]

"I AM A SONG  \r\n  \r\nI'm a song and I've waited so long  \r\nFor someone to come and sing me  \r\nI got a rhyme that I've had for some time  \r\nNobody wants to sing me  \r\n  \r\nLet me show, the world, that I'm, not through  \r\nLet me touch, your heart, the way I used to do  \r\n  \r\nSing me, sing me, now I got a song to be sung  \r\nSing me, sing me, roll me around on your tongue  \r\n  \r\nI got sunshine, I got a message to shout  \r\nSing me, sing me, won't someone please let me out  \r\n  \r\nCould I make you smile if I came back in style?  \r\nTell me would you want to sing me?  \r\nDon't say I'm gone, never more to live on  \r\nSomebody's got to sing me  \r\n  \r\nLet me fill , the air, with my, old song  \r\nIf I let, you down, tell me where did I go wrong  \r\n  \r\nSing me, sing me, I got a song to be sung (yes I do)  \r\nSing me, sing me, roll me around on your tongue  \r\n  \r\nI got sunshine, I got a message to shout  \r\nSing me, sing me, won't someone please let me

In [76]:
df.shape


(5000, 3)

Text Cleaning/Text Processing

In [77]:
df['text'].str.lower().replace(r'\w\s','').replace(r'\n',' ',regex=True)

0       i am a song  \r   \r i'm a song and i've waite...
1       i gotta take this moment  \r just to push you ...
2       and through all alex's wondrous  \r childhood ...
3       'there's a place for us, somewhere a time and ...
4       you tell me that it's over  \r i'm just someon...
                              ...                        
4995    i'm looking to beat hell, looks like i've got ...
4996    don't cry, don't cry, baby  \r don't cry, baby...
4997    i break down, fear is sinking in  \r the cold ...
4998    it's four in the morning, the end of december ...
4999    they got nothing on me anymore,  \r they got n...
Name: text, Length: 5000, dtype: object

In [78]:
import nltk
from nltk.stem.porter import PorterStemmer

In [79]:
stemmer = PorterStemmer()

In [80]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/avantikagupta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [81]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
tokenizer = TreebankWordTokenizer()

def token(txt):
    tokens = tokenizer.tokenize(txt)
    a = [stemmer.stem(w) for w in tokens]
    return " ".join(a)

print(token("you are beautiful, beauty"))


you are beauti , beauti


In [82]:
df['text'].apply(lambda x: token(x))

0       i am a song i 'm a song and i 've wait so long...
1       i got ta take thi moment just to push you down...
2       and through all alex 's wondrou childhood he w...
3       'there 's a place for us , somewher a time and...
4       you tell me that it 's over i 'm just someon y...
                              ...                        
4995    i 'm look to beat hell , look like i 've got t...
4996    do n't cri , do n't cri , babi do n't cri , ba...
4997    i break down , fear is sink in the cold come ,...
4998    it 's four in the morn , the end of decemb i '...
4999    they got noth on me anymor , they got noth on ...
Name: text, Length: 5000, dtype: object

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [84]:
tfid=TfidfVectorizer(analyzer='word', stop_words='english')

In [85]:
matrix=tfid.fit_transform(df['text'])

In [86]:
similar=cosine_similarity(matrix)

In [87]:
similar[0]

array([1.00000000e+00, 8.84898057e-04, 0.00000000e+00, ...,
       1.38536638e-02, 3.24820832e-02, 3.16014513e-01], shape=(5000,))

In [89]:
df[df['song']=='Not Alone'].index[0]

np.int64(4997)

Recommender Function

In [92]:
def recommender(song_name):
    idx= df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song=[]
    for s_id in distance[1:20]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [93]:
recommender("Not Alone")

['Giving In',
 'Almost Home',
 'We Gotta Live Together',
 'Home',
 'Hammer And Nails',
 'Something About Space Dude',
 'Hiding My Heart',
 'Pushing The Needle Too Far',
 'No Place To Go But Home',
 'Joan Of Arc',
 'Just Another Love Song',
 'Love Was Here',
 'I Was Young When I Left Home',
 "Maybe You're Right",
 'Busted Stuff',
 'Closing Time',
 'Leaving On A Jetplane',
 'Take Me Home',
 'Faith In Love']

In [94]:
import pickle

In [97]:
pickle.dump(similar,open("similarity", "wb"))

In [98]:
pickle.dump(df, open("df", "wb"))